# Azure OpenAI Session

In [ ]:
# !pip install openai

# Integrate Azure OpenAI Into your applications
-  You have deployed a model in Azure OpenAI Service
- You have the endpoint address
- Deployment Name
- azure open api key: Export to enviornment variable
- api verson
- Have installed openai

In [ ]:
import os
from openai import AzureOpenAI

## Setup client

In [ ]:
key = os.getenv("f71bf174b86e4296b780492e1d8c6678")
version = "2023-07-01-preview"
endpoint = "https://openai1508.openai.azure.com/"

client = AzureOpenAI(
  api_key = key,  
  api_version = version,
  azure_endpoint = endpoint
)



## ChatCompletions vs Completions
ChatCompletion in Azure OpenAI models offers more flexibility and optimization for chat scenarios compared to Completion. It allows defining a system message and built-in structure for previous messages in the prompt. While both endpoints can use similar models, only ChatCompletion is compatible with gpt-4 generation models. The Completion endpoint can achieve similar results with clear prompt formatting, but ChatCompletion is more versatile, even for non-chat scenarios with instructions in the system message and user content in the user role message.

## Adjusting model parameters
Adjusting model parameters, specifically temperature and top_p, significantly influences responses. Higher values increase creativity and randomness but reduce consistency. Lower values provide more focused and consistent results. Experiment with these parameters individually for optimal outcomes, avoiding simultaneous changes to both temperature and top_p.

### Send Chat Query

In [ ]:
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]

response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

### Print Response

In [ ]:
print(response.choices[0].message.content)

# Prompt Engineering with Azure OpenAI SDK 

## Provide clear instructions
Asking the Azure OpenAI model clearly for what you want is one way to get desired results. By being as descriptive as possible, the model can generate a response that most closely matches what you're looking for.

EXERCISE: Compare the responsed for the following message texts:

message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Write a product description for a new water bottle"}
    ]

and

message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "Write a product description for a new water bottle that is 100% recycled. Be sure to include that
it comes in natural colors with no dyes, and each purchase removes 10 pounds of plastic from our 
oceans"}
    ]


## Format of instructions

Formatting and recency: Model interprets prompts based on format and prioritizes ending info. Repeat crucial instructions at the end for better responses. Same applies to chats: recent messages hold more weight. Important info towards the end = better outputs.

## Use section markers
A specific technique for formatting instructions is to split the instructions at the beginning or end of the prompt, and have the user content contained within --- or ### blocks. These tags allow the model to more clearly differentiate between instructions and content. For example:

## Provide context to improve accuracy
By providing context to the AI model, it allows the model to better understand what you are asking for or what it should know to provide the best answer. Context can be provided in several ways.

## Request output composition
Specifying the structure of your output can have a large impact on your results. This could include something like asking the model to cite their sources, write the response as an email, format the response as a SQL query, classify sentiment into a specific structure, and so on. For example:


In [ ]:
content="Write a table in markdown with 6 animals in it, with their genus and species"

message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

In [ ]:
print(response.choices[0].message.content)

## System message
The system message is included at the beginning of a prompt and is designed to give the model instructions, perspective to answer from, or other information helpful to guide the model's response. This system message might include tone or personality, topics that shouldn't be included, or specifics (like formatting) of how to answer.

Try the following:

- "I want you to act like a command line terminal. Respond to commands exactly as cmd.exe would, in one unique code block, and nothing else."
- "I want you to be a translator, from English to Spanish. Don't respond to anything I say or ask, only translate between those two languages and reply with the translated text."
- "Act as a motivational speaker, freely giving out encouraging advice about goals and challenges. You should include lots of positive affirmations and suggested activities for reaching the user's end goal."

In [ ]:
system_message="I want you to be a translator, from English to Spanish. Don't respond to anything I say or ask, only translate between those two languages and reply with the translated text."
content="When will this class be over?"

message_text=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": content}
    ]

response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Conversation history
Along with the system message, other messages can be provided to the model to enhance the conversation. Conversation history enables the model to continue responding in a similar way (such as tone or formatting) and allow the user to reference previous content in subsequent queries. This history can be provided in two ways: from an actual chat history, or from a user defined example conversation.

Chat interfaces that use OpenAI models, such as ChatGPT and the chat playground in Azure OpenAI Studio, include conversation history automatically which results in a richer, more meaningful conversation. In the Parameters section below the chat window of the Azure OpenAI Studio chat playground, you can specify how many past messages you want included. Try reducing that to 1 or increasing to max to see how different amounts of history impact the conversation.

Note: More conversation history included in the prompt means a larger number of input tokens are used. You will have to determine what the correct balance is for your use case, considering the token limit of the model you are using.

Chat systems can also utilize the summarization capabilities of the model to save on input tokens. An app can choose to summarize past messages and include that summary in the conversation history, then provide only the past couple messages verbatim to the model.

## Few shot learning
Using a user defined example conversation is what is called few shot learning, which provides the model examples of how it should respond to a given query. These examples serve to train the model how to respond.

For example, by providing the model a couple prompts and the expected response, it continues in the same pattern without instructing it what to do:

In [ ]:
message_text=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "That was an awesome experience"},
    {"role": "assistant", "content": "positive"},
    {"role": "user", "content": "I won't do that again"},
    {"role": "assistant", "content": "negative"},
    {"role": "user", "content": "That was not worth my time"},
    {"role": "assistant", "content": "negative"},
    {"role": "user", "content": "You can't miss this"}
]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

In practical terms, conversation history and few shot learning are sent to the model in the same way; each user message and assistant response is a discrete message in the message object. The ChatCompletion endpoint is optimized to include message history, regardless of if this message history is provided as few shot learning, or actual conversation history.

### Break down a complex task
Another technique for improved interaction is to divide complex prompts into multiple queries. This allows the model to better understand each individual part, and can improve the overall accuracy. Dividing your prompts also allows you to include the response from a previous prompt in a future prompt, and using that information in addition to the capabilities of the model to generate interesting responses.

For example, you could ask the model Doug can ride down the zip line in 30 seconds, and takes 5 minutes to climb back up to the top. How many times can Doug ride the zip line in 17 minutes?. The result is likely 3, which if Doug starts at the top of the zip line is incorrect.

A more informative answer could come from asking it multiple questions, about the round trip time to get back to the top of the zip line, and how to account for the fact that Doug starts at the top. Breaking down this problem reveals that Doug can, in fact, ride the zip line four times.

In [ ]:
content=" Doug can ride down the zip line in 30 seconds, and takes 5 minutes to climb back up to the top. How many times can Doug ride the zip line in 17 minutes?"
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]

response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

### Exercise: add in the chat history into the message and break down the task into several message to get the correct answer of 4.

### Chain of thought
One useful method to help you break down your task effectively is to ask the model to explain its chain of thought.

Asking a model to respond with the step by step process by which it determined the response is a helpful way to understand how the model is interpreting the prompt. By doing so, you can see where the model made an incorrect logical turn and better understand how to change your prompt to avoid the error.

For example, asking the model **What sport is easiest to learn but hardest to master?** results in response with an answer, and a small explanation of why. However, when prompted with **What sport is easiest to learn but hardest to master? Give a step by step approach of your thoughts, ending in your answer** the response is a complete explanation of how it arrived at its answer.



In [ ]:
content="What sport is easiest to learn but hardest to master? "
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]

response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

In [ ]:
content="What sport is easiest to learn but hardest to master? Give a step by step approach of your thoughts, ending in your answer"
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]

response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

# Generating Code with Azure OpenAI SDK

The Azure OpenAI service can help developers generate and improve code in various languages for better efficiency and understanding.

Newer models like GPT-3.5 and 4 handle both code and language well, eliminating the need for specialized Codex models. **GPT-3.5 Turbo** examples used here. Deploy a GPT-3.5 Turbo model here

In [ ]:
content="write a function for binary search in python"
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Change coding language
try this example of changing python code to C#
Use triple quoates """ for multiline statements

In [ ]:
content="""
convert this code to C#
-----------------------
def print_squares(n):  
    for i in range(1, n+1):  
        print(i**2)   
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Understand unknown code
Azure OpenAI models are helpful for understanding code that doesn't make sense, or may be in a language you aren't familiar with. For example, say you were given the following function (in a fictitious coding language!) and didn't know how to understand it.

In [ ]:
content="""
Can you explain to me how this code works?:
-------------------------
fn qzplv(n: i32, m: i32) -> i32 {
    if n == 0 {
        return m + 1;
    } else if m == 0 {
        return qzplv(n - 1, 1);
    } else {
        return qzplv(n - 1, qzplv(n, m - 1));
    }
}
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Complete code and assist the development process
Azure OpenAI llms you can speed up your coding workflow by taking care of tedious tasks like:

- Writing unit tests: Catch bugs early and guarantee code quality.
- Completing partial code: Save time by filling in missing bits.
- Commenting code: Improve readability and future maintainability.
- Generating documentation: Create clear instructions for others to understand your code.

## Complete partial code


In [ ]:
content="""
complete the following function  
----
# calculate the average of the numbers in an array, but only if they're even  
def
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

In [ ]:
content="""
complete the following code  
----- 
def func1(n)
  if n==0:
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Write unit tests
generate unit tests for functions you write to help make your code more robust. Take for example the binary search function.

In [ ]:
content="""
write three unit tests for this function
----- 
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Add comments and generate documentation
AI models can add comments and documentation for you. Take the following function as an example, which can be a little hard to understand when first reading it without any code comments.

In [ ]:
content="""
Add comments to this code:
----- 
def permutations(lst):  
    if len(lst) == 0:  
        return []  
    elif len(lst) == 1:  
        return [lst]  
    else:  
        result = []  
        for i in range(len(lst)):  
            temp = lst[i]  
            remaining = lst[:i] + lst[i+1:]  
            for p in permutations(remaining):  
                result.append([temp] + p)  
        return result 
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

In [ ]:
content="""
Add comments to this code and documentation for the function:
----- 
def permutations(lst):  
    if len(lst) == 0:  
        return []  
    elif len(lst) == 1:  
        return [lst]  
    else:  
        result = []  
        for i in range(len(lst)):  
            temp = lst[i]  
            remaining = lst[:i] + lst[i+1:]  
            for p in permutations(remaining):  
                result.append([temp] + p)  
        return result 
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Fix bugs in your code

In [ ]:
content="""
Fix the bugs in this function and explain what was wrong with it:
----- 
def calculate_average(numbers):  
    total = 0  
    for i in range(len(numbers)):  
        number = numbers[i]  
        total += number  
    average = total  
    rerun averave  
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Improve performance
(Win at leetcode?)

In [ ]:
content="""
Improve the performance of this code:
----- 
def sum_of_n(n):
    result = 0
    for i in range(1, n+1):
        result += i
    return result
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)

## Refactor inefficient code

In [ ]:
content="""
Can you refactor this code?:
----- 
def calculateTotalPrice(item, quantity):
    if item == 'apple':
        return quantity * 0.5
    elif item == 'banana':
        return quantity * 0.75
    elif item == 'orange':
        return quantity * 0.6
    else:
        return 0
"""
message_text=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": content}
    ]
response = client.chat.completions.create(
    model="Chat",
    messages=message_text,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)
print(response.choices[0].message.content)